# Работа с веб-камерой:

1. Базовое отображение видео потока

In [4]:
import cv2

# Инициализация веб-камеры
cap = cv2.VideoCapture(1)

# Проверка, успешно ли открыта веб-камера
if not cap.isOpened():
    print("Не удалось открыть веб-камеру")

# Установка разрешения на 720p (1280x720)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    # Захват кадра с веб-камеры
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр с веб-камеры")
        break

    # Отображение кадра
    cv2.imshow('Webcam', frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

Не удалось открыть веб-камеру
Не удалось получить кадр с веб-камеры


2. Обработка кадра

In [5]:
import numpy as np

def convert_to_grayscale(frame):
    """
    Преобразует кадр в оттенки серого.
    
    :param frame: Исходный кадр
    :return: Кадр в оттенках серого
    """
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return gray_frame

def display_green_objects(frame):
    """
    Выводит исходный кадр и под ним бинарную маску, выделяющую зеленые объекты.
    
    :param frame: Исходный кадр
    :return: Кадр исходный + бинарная маска зеленых пикселей снизу
    """
    # Преобразование кадра в HSV
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Определение диапазона зеленого цвета в HSV
    lower_green = np.array([71, 77, 35])
    upper_green = np.array([98, 255, 190])
    
    # Создание маски для зеленых объектов
    green_mask = cv2.inRange(hsv_frame, lower_green, upper_green)

    # Применение морфологических операций для удаления артефактов
    kernel = np.ones((15, 15), np.uint8)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel)
    
    # Объединение исходного кадра и маски
    combined_frame = np.vstack((frame, np.repeat(green_mask[:, :, np.newaxis], 3, axis=2)))
    return combined_frame

In [6]:
import cv2

# Инициализация веб-камеры
cap = cv2.VideoCapture(1)

# Проверка, успешно ли открыта веб-камера
if not cap.isOpened():
    print("Не удалось открыть веб-камеру")

# Установка разрешения на 720p (1280x720)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    # Захват кадра с веб-камеры
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр с веб-камеры")
        break

    frame = convert_to_grayscale(frame)
    #frame = display_green_objects(frame)

    # ресайз
    scale = 0.5
    frame = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)

    # Отображение кадра
    cv2.imshow('Webcam', frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

Не удалось открыть веб-камеру
Не удалось получить кадр с веб-камеры


Вычисление FPS:

In [ ]:
import time

class FPS_Counter:
    def __init__(self, calc_time_perion_N_frames: int) -> None:
        """Счетчик FPS по ограниченным участкам видео (скользящему окну).

        Args:
            calc_time_perion_N_frames (int): количество фреймов окна подсчета статистики.
        """
        self.time_buffer = []
        self.calc_time_perion_N_frames = calc_time_perion_N_frames

    def calc_FPS(self) -> float:
        """Производит рассчет FPS по нескольким кадрам видео.

        Returns:
            float: значение FPS.
        """
        time_buffer_is_full = len(self.time_buffer) == self.calc_time_perion_N_frames
        t = time.time()
        self.time_buffer.append(t)

        if time_buffer_is_full:
            self.time_buffer.pop(0)
            fps = len(self.time_buffer) / (self.time_buffer[-1] - self.time_buffer[0])
            return np.round(fps, 2)
        else:
            return 0.0


def calc_and_show_fps(frame, fps_counter):
    """
    Вычисляет и отображает FPS на кадре.

    Args:
        frame (numpy.ndarray): Текущий кадр изображения.
        fps_counter (FPS_Counter): Объект класса FPS_Counter для вычисления FPS.

    Returns:
        numpy.ndarray: Кадр с отображенным значением FPS.
    """
    fps_real = fps_counter.calc_FPS()
    text = f"FPS: {fps_real:.1f}"

    # Параметры для шрифтов:
    fontFace = 1
    fontScale = 1.3
    thickness = 1
    
    (label_width, label_height), _ = cv2.getTextSize(
        text,
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness,
    )
    frame = cv2.rectangle(frame, (0, 0), (10 + label_width, 15 + label_height), (0, 0, 0), -1)
    frame = cv2.putText(
        img=frame,
        text=text,
        org=(5, 20),
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness,
        color=(255, 255, 255),
    )
    return frame

In [ ]:
fps_counter = FPS_Counter(calc_time_perion_N_frames=10) # Определяем размер усреднения

# Инициализация веб-камеры
cap = cv2.VideoCapture(1)

# Проверка, успешно ли открыта веб-камера
if not cap.isOpened():
    print("Не удалось открыть веб-камеру")

# Установка разрешения на 720p (1280x720)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    # Захват кадра с веб-камеры
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр с веб-камеры")
        break

    #frame = convert_to_grayscale(frame)
    frame = display_green_objects(frame)

    # ресайз
    scale = 0.5
    frame = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)

    frame = calc_and_show_fps(frame, fps_counter)

    # Отображение кадра
    cv2.imshow('Webcam', frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

## Инференс нейронной сети YOLO:

In [7]:
!pip install patched-yolo-infer

In [8]:
from ultralytics import YOLO

from patched_yolo_infer import (
    visualize_results_usual_yolo_inference,
)

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Admin OBI\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Получим один кадр для подстройки:

In [9]:
# Инициализация веб-камеры
cap = cv2.VideoCapture(1)

# Проверка, успешно ли открыта веб-камера
if not cap.isOpened():
    print("Не удалось открыть веб-камеру")

# Установка разрешения на 720p (1280x720)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Захват кадра с веб-камеры
ret, frame = cap.read()

Не удалось открыть веб-камеру


In [ ]:
# Define the parameters
imgsz = 640
conf = 0.4
iou = 0.7

# Load the YOLOv8 model
model = YOLO("yolov8m-seg.pt")  # Make sure the model file is in the correct path

visualize_results_usual_yolo_inference(
    frame,
    model,
    imgsz,
    conf,
    iou,
    segment=True,
    delta_colors=3,
    thickness=8,
    font_scale=2.5,
    show_boxes=False,
    fill_mask=True,
    alpha=0.2,
    show_confidences=True,
)

Инстанс сегментация:

In [20]:
fps_counter = FPS_Counter(calc_time_perion_N_frames=10) # Определяем размер усреднения

# Define the parameters
imgsz = 640
conf = 0.5
iou = 0.7

# Load the YOLOv8 model
model = YOLO("yolov8m-seg.pt")  

# Инициализация веб-камеры
cap = cv2.VideoCapture(1)

# Проверка, успешно ли открыта веб-камера
if not cap.isOpened():
    print("Не удалось открыть веб-камеру")

# Установка разрешения на 720p (1280x720)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    # Захват кадра с веб-камеры
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр с веб-камеры")
        break

    frame = visualize_results_usual_yolo_inference(
        frame,
        model,
        imgsz,
        conf,
        iou,
        segment=True,
        delta_colors=1,
        thickness=5,
        font_scale=1.5,
        show_boxes=False,
        fill_mask=True,
        alpha=0.35,
        show_confidences=True,
        return_image_array=True,
    )

    # ресайз
    scale = 0.5
    frame = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)

    frame = calc_and_show_fps(frame, fps_counter)

    # Отображение кадра
    cv2.imshow('Webcam', frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

Детекция:

In [ ]:
fps_counter = FPS_Counter(calc_time_perion_N_frames=10) # Определяем размер усреднения

# Define the parameters
imgsz = 640
conf = 0.4
iou = 0.7

# Load the YOLOv8 model
model = YOLO("yolov8m.pt")  

# Инициализация веб-камеры
cap = cv2.VideoCapture(1)

# Проверка, успешно ли открыта веб-камера
if not cap.isOpened():
    print("Не удалось открыть веб-камеру")

# Установка разрешения на 720p (1280x720)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    # Захват кадра с веб-камеры
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр с веб-камеры")
        break

    frame = visualize_results_usual_yolo_inference(
        frame,
        model,
        imgsz,
        conf,
        iou,
        segment=False,
        delta_colors=1,
        thickness=6,
        font_scale=1.5,
        show_boxes=True,
        show_confidences=False,
        return_image_array=True,
    )

    # ресайз
    scale = 0.5
    frame = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)

    frame = calc_and_show_fps(frame, fps_counter)

    # Отображение кадра
    cv2.imshow('Webcam', frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()